OpenClassrooms
Project 4, Data Scientist
Author : Oumeima EL GHARBI
Date : August 2022

Un notebook pour chaque prédiction (émissions de CO2 et consommation totale d’énergie) des différents tests de modèles mis au propre, dans lequel vous identifierez clairement le modèle final choisi.


L’objectif est de te passer des relevés de consommation annuels futurs (attention à la fuite de données). Nous ferons de toute façon pour tout nouveau bâtiment un premier relevé de référence la première année, donc rien ne t'interdit d’en déduire des variables structurelles aux bâtiments, par exemple la nature et proportions des sources d’énergie utilisées..

Fais bien attention au traitement des différentes variables, à la fois pour trouver de nouvelles informations (peut-on déduire des choses intéressantes d’une simple adresse ?) et optimiser les performances en appliquant des transformations simples aux variables (normalisation, passage au log, etc.).

Mets en place une évaluation rigoureuse des performances de la régression, et optimise les hyperparamètres et le choix d’algorithmes de ML à l’aide d’une validation croisée.



Tester les modèles suivants : **regression linéaire (avec différentes régularisation : Ridge, Lasso, Elastic), Random Forest, XGboost**
Penser à comparer les performances des différents modèles : utiliser la **MAE**
Penser également à optimiser les hyper paramètres de chaque modèle via **GridSearch**


Evaluate :

https://cloud.google.com/automl-tables/docs/evaluate?hl=fr


KFOLD

Entrée : données X (dimension nxp), étiquettes y (dimension n), nombre de folds k

Couper [0, 1, ..., n-1] en k parties de taille (n/k). (La dernière partie sera un peu plus petite si n n'est pas un multiple de k)

for i=0 to (k-1):
    Former le jeu de test (X_test, y_test) en restreignant X et y aux indices contenus dans la i-ième partie.
    Former le jeu d'entraînement (X_train, y_train) en restreignant X et y aux autres indices.
    Entraîner l'algorithme sur le jeu d'entraînement
    Utiliser le modèle ainsi obtenu pour prédire sur le jeu de test
        Calculer l'erreur du modèle en comparant les étiquettes prédites aux vraies étiquettes contenues dans y_test

Sortie : la valeur moyenne des erreurs calculées sur les k folds.

In [3]:
# 1 modele de Regression (RL classsque / Elastic / ridig / laso
# Random Forest
# XGBOOST

# var à rpedire tottal GHE Emssion last one to predict
# cette var deped de la consommation des bateimenst (1) prediction sur elec, steam, naturalgas et un autre energie (2) et reutiliser pour predire

### Introduction

#### Importing libraries

In [4]:
%reset -f

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline
%autosave 300

Autosaving every 300 seconds


#### Loading dataset

In [5]:
columns_to_categorize = ["BuildingType", "PrimaryPropertyType", "ZipCode", "CouncilDistrictCode", "LargestPropertyUseType", "SecondLargestPropertyUseType", "ThirdLargestPropertyUseType"]

category_types = {column: 'category' for column in columns_to_categorize}
print("This dictionary will be used when reading the csv file to assign a type to categorical features :", category_types)

This dictionary will be used when reading the csv file to assign a type to categorical features : {'BuildingType': 'category', 'PrimaryPropertyType': 'category', 'ZipCode': 'category', 'CouncilDistrictCode': 'category', 'LargestPropertyUseType': 'category', 'SecondLargestPropertyUseType': 'category', 'ThirdLargestPropertyUseType': 'category'}


In [6]:
dataset_path = "dataset/2016_Building_Energy_Prediction.csv"
# we assign the categorical features with a categotical type
data = pd.read_csv(dataset_path, dtype=category_types)

In [7]:
data.shape

(3157, 23)

In [8]:
data.dtypes

DataYear                              int64
BuildingType                       category
PrimaryPropertyType                category
ZipCode                            category
CouncilDistrictCode                category
YearBuilt                             int64
NumberofBuildings                   float64
NumberofFloors                        int64
PropertyGFATotal                      int64
PropertyGFAParking                    int64
PropertyGFABuilding(s)                int64
LargestPropertyUseType             category
LargestPropertyUseTypeGFA           float64
SecondLargestPropertyUseType       category
SecondLargestPropertyUseTypeGFA     float64
ThirdLargestPropertyUseType        category
ThirdLargestPropertyUseTypeGFA      float64
ENERGYSTARScore                     float64
SiteEnergyUse(kBtu)                 float64
SteamUse(kBtu)                      float64
Electricity(kBtu)                   float64
NaturalGas(kBtu)                    float64
TotalGHGEmissions               

In [9]:
# predict : Electricity
# train / test
# standardisation = retirer la moyen et div par ecart type (scaling : les var qn sur emem echelle
# var categ : encoding (one hot encoder)

# la fin Feature engineriing

# 2) entrainer le smodels
# perf
# temps de calcul
# graph pour montrer la perf de chaque modele(barplot)
# obj : finir exploration / finir feature engineering
# obj un premier noteboook propre (try max)

### I) Feature Engineering : preparing the vectors and matrices


#### 1) Splitting training / test dataset

In [10]:
y = data.copy()[{'SiteEnergyUse(kBtu)', 'TotalGHGEmissions'}]
X = data.copy().drop(['SiteEnergyUse(kBtu)', 'TotalGHGEmissions'], axis=1)

#### 2) Normalization and One Hot Encoder

In [11]:
X.select_dtypes(['category','object']).nunique() # we don't have 'object' here but it is just in case.

BuildingType                     8
PrimaryPropertyType             23
ZipCode                         54
CouncilDistrictCode              7
LargestPropertyUseType          55
SecondLargestPropertyUseType    51
ThirdLargestPropertyUseType     45
dtype: int64

#### 3) Separation train and test dataset


In [2]:
print("DO LIKE BELOW  :")

DO LIKE BELOW  :
DO LIKE BELOW  :


In [ ]:
import numpy as np
# charger les données
import pandas as pd

data = pd.read_csv('', sep=';')
features_for_prediction = []
variable_to_predict = ''

# créer la matrice de données
X = data[data.columns[features_for_prediction]].values

# créer le vecteur d'étiquettes
y = data[variable_to_predict].values


In [11]:
from sklearn import model_selection
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.3)


In [11]:
# standardiser les données
from sklearn import preprocessing
std_scale = preprocessing.StandardScaler().fit(X_train)

X_train_std = std_scale.transform(X_train)
X_test_std = std_scale.transform(X_test)

### II) Modelisation

In [12]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error

In [13]:
variable_to_predict = "Electricity(kBtu)"

### 1) Linear Regression / Ridge Regression / Lasso / Elastic Net


In [14]:
X = data.copy()
y = data.copy()

de standardiser les variables avant d'apprendre une régression ridge. Il s'agit de faire en sorte que chaque variable ait un écart-type de 1, en leur appliquant la transformation suivante :

Xij←Xij1n∑ni=1(Xij−1n∑ni=1Xij)2−−−−−−−−−−−−−−−−−−−−−√.
 Cette transformation est implémentée dans sklearn.preprocessing.StandardScaler.

In [15]:
from sklearn import preprocessing

std_scale = preprocessing.StandardScaler().fit(X)
X_scale = std_scale.transform(X)

ValueError: could not convert string to float: 'Multifamily LR (1-4)'

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_scale, y, test_size=0.25)

In [ ]:
X_train.shape

In [ ]:
X_test.shape

#### 1) Linear Regression : baseline

In [ ]:
from sklearn import linear_model

# On crée un modèle de régression linéaire
lr = linear_model.LinearRegression()
# On entraîne ce modèle sur les données d'entrainement
lr.fit(X_train, y_train)

In [ ]:
# On récupère l'erreur de norme 2 sur le jeu de données test comme baseline
baseline_error = np.mean((lr.predict(X_test) - y_test) ** 2)

#On obtient l'erreur quadratique ci-dessous
print(baseline_error)

#### 2) Linear Regression : Ridge

In [ ]:
n_alphas = 50 #hyperparametre alpha
alphas = np.logspace(-5, 5, 50)

ridge = linear_model.Ridge()

coefs = []
errors = []
for a in alphas:
    ridge.set_params(alpha=a)
    ridge.fit(X_train)
    coefs.append(ridge.coef_)
    errors.append(np.mean((ridge.predict(X_test) - y_test) ** 2))

In [ ]:
# observation du comportement de l'erreur

ax = plt.gca()
ax.plot(alphas, errors, [10**-5, 10**5], [baseline_error, baseline_error])
ax.set_xscale('log')
plt.show()

In [ ]:
ax = plt.gca()

ax.plot(alphas, errors)
ax.set_xscale('log')
plt.xlabel('alpha')
plt.ylabel('error')
plt.axis('tight')
plt.show()

In [ ]:
# index du min des erreurs
np.argmin(errors)

In [ ]:
# recupere l'erreur min
errors[np.argmin(errors)]

In [ ]:
# recup alpha associé à cet erreur min
alphas[np.argmin(errors)]
# alphas[35]

In [ ]:
# chemin de régularisation
ax = plt.gca()
ax.plot(alphas, coefs)
ax.set_xscale('log')
plt.show()

In [ ]:
ax = plt.gca()

ax.plot(alphas, coefs)
ax.set_xscale('log')
plt.xlabel('alpha')
plt.ylabel('weights')
plt.title('Ridge coefficients as a function of the regularization')
plt.axis('tight')
plt.show()

In [ ]:
min(errors)

#### 3) Linear Regression : LASSO

In [ ]:
n_alphas = 50
alphas = np.logspace(-5, 5, 50)

lasso = linear_model.Lasso()

coefs = []
errors = []
for a in alphas:
    lasso.set_params(alpha=a)
    lasso.fit(X_train)
    coefs.append(lasso.coef_)
    errors.append(np.mean((lasso.predict(X_test) - y_test) ** 2))

In [ ]:
n_alphas = 300
alphas = np.logspace(-5, 1, n_alphas)
lasso = linear_model.Lasso(fit_intercept=False)

"""
fit_intercept : bool, default=True
Whether to calculate the intercept for this model. If set to False, no intercept will be used in calculations (i.e. data is expected to be centered).
"""

coefs = []
errors = []
for a in alphas:
    lasso.set_params(alpha=a)
    lasso.fit(X_train, y_train)
    coefs.append(lasso.coef_)
    errors.append([baseline_error, np.mean((lasso.predict(X_test) - y_test) ** 2)])

In [ ]:
# observation du comportement de l'erreur

ax = plt.gca()
ax.plot(alphas, errors, [10**-5, 10**5], [baseline_error, baseline_error])
ax.set_xscale('log')
plt.show()

In [ ]:
# recupere l'erreur min
errors[np.argmin(errors)]

In [ ]:
# recup alpha associé à cet erreur min
alphas[np.argmin(errors)]

In [ ]:
# chemin de régularisation
ax = plt.gca()
ax.plot(alphas, coefs)
ax.set_xscale('log')
plt.show()

In [ ]:
ax = plt.gca()

ax.plot(alphas, coefs)
ax.set_xscale('log')
plt.xlabel('alpha')
plt.ylabel('weights')
plt.axis('tight')
plt.show()

In [22]:
import numpy as np

X = np.matrix([-0.78768, -1.51760513, 0.74416271, -0.62288928])
X = X.T
print(X)

y = np.matrix([-34.59703199, -30.79543532, 19.31018182, -19.44809959])
y = y.T
print(y)
print(np.linalg.inv(X.T.dot(X)).dot(X.T))
beta = np.linalg.inv(X.T.dot(X)).dot(X.T).dot(y)
beta

[[-0.78768   ]
 [-1.51760513]
 [ 0.74416271]
 [-0.62288928]]
[[-34.59703199]
 [-30.79543532]
 [ 19.31018182]
 [-19.44809959]]
[[-0.20378056 -0.39261937  0.19252221 -0.16114758]]


matrix([[25.99274029]])

#### Export des modèles pour réutilisation ultérieure


#### Chargement des modèles


#### Comparaison des modèles


### III) Evaluation

#### Vérification des prédictions


#### Intérêt du Energy Star Score
